<a href="https://colab.research.google.com/github/Lakshay084/Mandi_Selection/blob/main/Mandi_Selection_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Best APMC Market top sale FarmProduce

In [ ]:
class Mandi:
    def __init__(self, name, distance, rate):
        self.name = name
        self.distance = distance
        self.rate = rate

    def transportation_cost(self, quantity):
        if quantity <= 30:
            if 1 <= self.distance <= 10:
                return 1000
            elif 10 < self.distance <= 20:
                return 1500
            elif 20 < self.distance <= 40:
                return 2500
            elif 40 < self.distance <= 100:
                return 5000
        elif 30 < quantity <= 50:
            if 1 <= self.distance <= 10:
                return 2000
            elif 10 < self.distance <= 20:
                return 2500
            elif 20 < self.distance <= 40:
                return 4000
            elif 40 < self.distance <= 100:
                return 5000
        elif 50 < quantity <= 100:
            if 1 <= self.distance <= 10:
                return 3000
            elif 10 < self.distance <= 20:
                return 3000
            elif 20 < self.distance <= 40:
                return 5000
            elif 40 < self.distance <= 100:
                return 7000
        elif 100 < quantity <= 150:
            if 1 <= self.distance <= 10:
                return 5000
            elif 10 < self.distance <= 20:
                return 5000
            elif 20 < self.distance <= 40:
                return 6000
            elif 40 < self.distance <= 100:
                return 9000
        return 0

    def loading_unloading_charge(self, quantity):
        if self.distance != 0:
            return (0.12 + 0.1) * quantity * 100  # converting quintal to kg
        else:
            return 0

    def extra_charges(self):
        if self.distance != 0:
            return 200
        else:
            return 0

    def total_cost(self, quantity):
        return self.transportation_cost(quantity) + self.loading_unloading_charge(quantity) + self.extra_charges()

    def net_profit(self, quantity):
        revenue = self.rate * quantity * 100  # converting quintal to kg
        cost = self.total_cost(quantity)
        return revenue - cost


class SuggestionModel:
    def __init__(self, mandis):
        self.mandis = mandis

    def suggest_best_mandi(self, quantity):
        profits = {}
        costs = {}
        best_mandi = None
        max_profit = float('-inf')

        for mandi in self.mandis:
            profit = mandi.net_profit(quantity)
            cost = mandi.total_cost(quantity)
            profits[mandi.name] = profit
            costs[mandi.name] = cost
            if profit > max_profit:
                max_profit = profit
                best_mandi = mandi

        return best_mandi, max_profit, profits, costs


# Creating Mandi objects
kota_mandi = Mandi("Kota Mandi", 100, 50)
baran_mandi = Mandi("Baran Mandi", 45, 49.8)
khanpur_mandi = Mandi("Khanpur Mandi", 45, 49.7)
bapawar_mandi = Mandi("Bapawar Mandi", 12, 49.5)
village = Mandi("From Village", 0, 49.1)

# List of all mandis
mandis = [kota_mandi, baran_mandi, khanpur_mandi, bapawar_mandi, village]

# Create a SuggestionModel
suggestion_model = SuggestionModel(mandis)

# Farmer's quantity to sell (in quintals) - input from the user
while True:
    quantity = float(input("Enter the quantity to sell (in quintals, max 150 quintals): "))
    if 0 < quantity <= 150:
        break
    else:
        print("Please enter a valid quantity between 1 and 150 quintals.")

# Get the best mandi suggestion and profit comparison
best_mandi, max_profit, profits, costs = suggestion_model.suggest_best_mandi(quantity)

# Display the result
print(f"\nBest mandi to sell is: {best_mandi.name}")
print(f"Expected profit at best mandi: ₹{max_profit:.2f}\n")

# Display the profit for all mandis
print("Profit comparison for all mandis:")
for mandi_name, profit in profits.items():
    cost = costs[mandi_name]
    profit_difference = max_profit - profit
    print(f"{mandi_name}: ₹{profit:.2f} (Difference: ₹{profit_difference:.2f} cost: {costs[mandi_name]:.2f})")


Enter the quantity to sell (in quintals, max 150 quintals): 80

Best mandi to sell is: From Village
Expected profit at best mandi: ₹392800.00

Profit comparison for all mandis:
Kota Mandi: ₹391040.00 (Difference: ₹1760.00 cost: 8960.00)
Baran Mandi: ₹389440.00 (Difference: ₹3360.00 cost: 8960.00)
Khanpur Mandi: ₹388640.00 (Difference: ₹4160.00 cost: 8960.00)
Bapawar Mandi: ₹391040.00 (Difference: ₹1760.00 cost: 4960.00)
From Village: ₹392800.00 (Difference: ₹0.00 cost: 0.00)


Sale Time Decision.

In [ ]:
class SoybeanSaleDecision:
    def __init__(self, initial_price_oct, initial_price_nov, initial_moisture_oct, final_moisture_nov):
        self.initial_price_oct = initial_price_oct
        self.initial_price_nov = initial_price_nov
        self.initial_moisture_oct = initial_moisture_oct
        self.final_moisture_nov = final_moisture_nov

    def calculate_price_adjustment(self, moisture, base_price):
        if 10 <= moisture <= 12:
            adjusted_price = base_price - (moisture - 10) * base_price / 100
        elif 12 < moisture <= 15:
            adjusted_price = base_price - ((moisture - 12) * base_price * 2 / 100) - (3 * base_price / 100)
        else:
            adjusted_price = base_price - ((moisture - 15) * base_price * 2.5 / 100) - (3 * base_price * 2 / 100) - (3 * base_price / 100)
        return adjusted_price

    def calculate_weight_loss(self, initial_weight, initial_moisture, final_moisture):
        weight_loss_percentage = (initial_moisture - final_moisture) / 100
        final_weight = initial_weight * (1 - weight_loss_percentage)
        return final_weight

    def calculate_profit(self, quantity, price, weight):
        return price * weight * quantity  # Quantity in quintals, weight in kg

    def suggest_sale_time(self, initial_weight, quantity):
        # Calculate October profit
        adjusted_price_oct = self.calculate_price_adjustment(self.initial_moisture_oct, self.initial_price_oct)
        profit_oct = self.calculate_profit(quantity, adjusted_price_oct, initial_weight)

        # Calculate November profit
        final_weight_nov = self.calculate_weight_loss(initial_weight, self.initial_moisture_oct, self.final_moisture_nov)
        adjusted_price_nov = self.calculate_price_adjustment(self.final_moisture_nov, self.initial_price_nov)
        profit_nov = self.calculate_profit(quantity, adjusted_price_nov, final_weight_nov)

        # Calculate the difference
        profit_difference = profit_nov - profit_oct

        return profit_oct, profit_nov, profit_difference

# Initialize the conditions
initial_price_oct = 45  # Price for 10% moisture in October
initial_price_nov = 50  # Price for 10% moisture in November
initial_moisture_oct = 15  # Moisture content in October
final_moisture_nov = 10  # Moisture content in November

# Initialize the decision-making class
soybean_decision = SoybeanSaleDecision(initial_price_oct, initial_price_nov, initial_moisture_oct, final_moisture_nov)

# Input the quantity and initial weight per kg for the calculation
quantity = float(input("Enter the quantity to sell (in quintals): "))
initial_weight = 100  # 100 kg per quintal

# Calculate profits and difference
profit_oct, profit_nov, profit_difference = soybean_decision.suggest_sale_time(initial_weight, quantity)

# Display the results
print(f"October Profit: ₹{profit_oct:.2f}")
print(f"November Profit: ₹{profit_nov:.2f}")
print(f"Difference: ₹{profit_difference:.2f}")

if profit_difference > 0:
    print("It is better to sell in November.")
else:
    print("It is better to sell in October.")


Enter the quantity to sell (in quintals): 120
October Profit: ₹491400.00
November Profit: ₹570000.00
Difference: ₹78600.00
It is better to sell in November.
